In [5]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
from scipy.optimize import minimize
from cvxopt import matrix, solvers
from sklearn.model_selection import KFold, train_test_split
from scipy.spatial.distance import cdist
import logging
import time

In [6]:
data = pd.read_csv('data.txt')

In [7]:
x = data.drop(['Y'], axis=1).to_numpy()
y = (data['Y'] == 'P').astype(int).to_numpy()
y = np.column_stack((y==0, y==1)).astype(int)

In [8]:
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
# Activation functions

class Softmax:
    def __call__(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
    
    def grad(self, sm_x):
        return sm_x * (1 - sm_x)

class Sigmoid:
    def __call__(self, x):
        return 1 / (1 + np.exp(-x)) 
        
    def grad(self, s_x):
        return s_x * (1 - s_x)

class Tanh:
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, x):
        return np.tanh(self.sigma * x)

    def grad(self, th_x):
        return self.sigma * (1 - th_x ** 2)
    
class Gaussian():
    def __init__(self, sigma=1, gamma=1):
        self.sigma = sigma

    def __call__(self, x):
        return np.exp(- x ** 2 / self.sigma ** 2)

    def derivative(self, x):
        return - 2 * x / self.sigma ** 2 * self.__call__(x)

# Multilayer Perceptron (MLP)

In [117]:
# Layer of neural network

class MLPLayer():
    def __init__(self, input_size, output_size, activation):
        self.w = (np.random.random((input_size, output_size)) - 0.5) * 2
        self.b = (np.random.random(output_size) - 0.5) * 2
        self.activation = activation
        self.input = None
        self.output = None
        self.grad_w = None
        self.grad_b = None
        self.grad_input = None

    def Forward(self, input):
        self.input = input
        sum = input @ self.w + self.b
        self.output = self.activation(sum)

    def Backward(self, grad_output):
        a = self.activation.grad(self.output) * grad_output
        self.grad_b = a.sum(axis=0)
        self.grad_w = self.input.T @ a
        self.grad_input = a @ self.w.T

In [140]:
# Neural network

class MLP():
    def __init__(self, n, N, H, sigma=1):
        self.n = n
        self.rho = 1e-4
        self.layers = [MLPLayer(n, N, Tanh(sigma))]
        for i in range(H-1):
            self.layers.append(MLPLayer(N, N, Tanh(sigma)))
        self.layers.append(MLPLayer(N, 2, Sigmoid()))

    def get_omega(self):
        return np.concatenate([np.concatenate((layer.w.reshape(-1), layer.b)) for layer in self.layers])
        
    def update_omega(self, omega):
        i = 0
        for layer in self.layers:
            layer.w = omega[i : i + layer.w.size].reshape(layer.w.shape)
            i += layer.w.size
            layer.b = omega[i : i + layer.b.size]
            i += layer.b.size

    def predict(self, x):
        z = x
        for layer in self.layers:
            layer.Forward(z)
            z = layer.output
        return z

    def loss(self, x, y):
        p = self.predict(x)
        loss = - (np.sum(y * np.log(p), axis=1) + np.sum((1-y) * np.log(1-p), axis=1)).mean()
        loss += self.rho * np.linalg.norm(self.get_omega()) ** 2
        return loss

    def gradient(self, x, y):
        p = self.predict(x)
        last_grad = - (y / p - (1-y) / (1-p)) / x.shape[0]
        for layer in self.layers[::-1]:
            layer.Backward(last_grad)
            last_grad = layer.grad_input
        grad = np.concatenate([np.concatenate((layer.grad_w.reshape(-1), layer.grad_b)) for layer in self.layers])
        grad += 2 * self.rho * self.get_omega()
        return grad

    def accuracy(self, x, y):
        p = self.predict(x)
        return (np.sum(y*p, axis=1)).mean()

    def fit(self, x, y, method='trust-constr'):
        def fun(omega):
            self.update_omega(omega)
            return self.loss(x, y)

        def jac(omega):
            self.update_omega(omega)
            return self.gradient(x, y)

        omega0 = self.get_omega()
        res = minimize(fun=fun, jac=jac, x0=omega0, method=method, tol=1e-6, options={'maxiter': 1000})
        self.update_omega(res.x)
        return self.accuracy(x, y)

In [ ]:
# Cross-validation

models_data = []
for H in tqdm(range(1, 5)):
    for N in tqdm(range(1, 10, 2)):
        for log_sigma in tqdm(range(1, 5)):
            sigma = 10 ** log_sigma
            kf = KFold(n_splits=5, shuffle=True)
            kf.get_n_splits(x_train_val)

            for train_index, valid_index in kf.split(x_train_val):

                x_train = x_train_val[train_index]
                y_train = y_train_val[train_index]
                x_val = x_train_val[valid_index]
                y_val = y_train_val[valid_index]

                start = time.time()
                model = MLP(x_train.shape[1], N, H, sigma)
                model.fit(x_train, y_train)
                end = time.time()

                train_error = model.loss(x_train, y_train)
                val_error = model.loss(x_val, y_val)
                test_error = model.loss(x_test, y_test)

                models_data.append({'H': H,
                                    'N': N,
                                    'sigma': sigma,
                                    'train_error': train_error,
                                    'val_error': val_error,
                                    'test_error': test_error,
                                    'time': end - start})

In [123]:
# Results

models_data = pd.DataFrame(models_data)
models_data

,H,N,sigma,train_error,val_error
0,1,1,10,0.098969,0.115250
1,1,1,10,0.104970,0.090720
2,1,1,10,0.101427,0.105488
3,1,1,10,0.101478,0.105494
4,1,1,10,0.102515,0.100936
...,...,...,...,...,...
395,4,9,10000,0.298665,0.316079
396,4,9,10000,0.264039,0.269047
397,4,9,10000,0.309545,0.330924
398,4,9,10000,0.220962,0.196703


In [127]:
# Best model

grouped = models_data.groupby(['H', 'N', 'sigma']).agg('mean').reset_index()
idx = grouped.val_error.idxmin()
grouped.loc[idx]

H               1.000000
N               7.000000
sigma          10.000000
train_error     0.060798
val_error       0.075699
Name: 12, dtype: float64

# Radial Basis Function (RBF) Network

In [10]:
# Data preprocessing

y_train_val = y_train_val[:, 1]
y_test = y_test[:, 1]

In [25]:
# RBF network

class RBF_network():
    
    def __init__(self, x, N, sigma):
        self.w = (np.random.random(N) - 0.5) * 2
        self.c = np.random.random((N, x.shape[1])) * 15
        self.kernel = Gaussian(sigma)
        self.d = self.distance(x)

    def distance(self, x):
        return cdist(x, self.c, metric='euclidean')
        
    def predict(self, x):
        phi = self.kernel(self.distance(x))
        y_hat = phi @ self.w.T
        return (y_hat>0).astype(int)

    def loss(self, x, y):
        y_hat = self.predict(x)
        loss = ((y-y_hat)**2).mean()
        return loss
    
    def grad_w(self, x, y):
        phi = self.kernel(self.distance(x))
        y_hat = self.predict(x)
        grad_w = phi.T @ (y_hat - y) 
        return grad_w
        
    def grad_c(self, x, y):
        difference = x[:, None, :] - self.c[None, :, :]
        d = self.distance(x)
        y_hat = self.predict(x)
        grad_c = - np.einsum('i,j,ij,ijt,ij->jt', y-y_hat, self.w, self.kernel.derivative(d), difference, 1/d)
        return grad_c.reshape(-1)    

    def accuracy(self, x, y):
        y_hat = self.predict(x)
        return (y == y_hat).mean()

    def fit(self, x, y, method='trust-constr'):
        
        def fun_w(w):
            self.w = w
            return self.loss(x, y)
        
        def fun_c(c):
            self.c = c.reshape(self.c.shape)
            return self.loss(x, y)
        
        def jac_w(w):
            self.w = w
            return self.grad_w(x, y)

        def jac_c(c):
            self.c = c.reshape(self.c.shape)
            return self.grad_c(x, y)

        tol = 1e-6
        pred_error = self.loss(x, y)
        while True:
            res = minimize(fun=fun_c, jac=jac_c, x0=self.c.reshape(-1), method=method, tol=tol)
            self.c = res.x.reshape(self.c.shape)
            res = minimize(fun=fun_w, jac=jac_w, x0=self.w, method=method, tol=tol)
            self.w = res.x
            if abs(pred_error - res.fun) < tol:
                break
            pred_error = res.fun
        return self.accuracy(x, y)
    

In [ ]:
# Cross-validation
models_data = []
for N in tqdm(range(1, 5)):
    for log_sigma in tqdm(range(1, 5)):
        sigma = 10 ** log_sigma
        kf = KFold(n_splits=5, shuffle=True)
        kf.get_n_splits(x_train_val)

        for train_index, valid_index in kf.split(x_train_val):

            x_train = x_train_val[train_index]
            y_train = y_train_val[train_index]
            x_val = x_train_val[valid_index]
            y_val = y_train_val[valid_index]

            start = time.time()
            model = RBF_network(x_train, N, sigma)
            model.fit(x_train, y_train)
            end = time.time()

            train_error = model.loss(x_train, y_train)
            val_error = model.loss(x_val, y_val)
            test_error = model.loss(x_test, y_test)

            models_data.append({'N': N,
                                'sigma': sigma,
                                'train_error': train_error,
                                'val_error': val_error,
                                'test_error': test_error,
                                'time': end - start})

In [64]:
# Results

models_data = pd.DataFrame(models_data)
models_data

,N,sigma,train_error,val_error
0,1,10,0.038906,0.041250
1,1,10,0.041484,0.030937
2,1,10,0.039219,0.040000
3,1,10,0.037812,0.045625
4,1,10,0.039453,0.039062
...,...,...,...,...
75,4,10000,0.039375,0.039375
76,4,10000,0.038984,0.040938
77,4,10000,0.038906,0.041250
78,4,10000,0.039844,0.037500


In [66]:
# Best model

grouped = models_data.groupby(['N', 'sigma']).agg('mean').reset_index()
idx = grouped.val_error.idxmin()
grouped.loc[idx]

N               4.000000
sigma          10.000000
train_error     0.037937
val_error       0.037250
Name: 12, dtype: float64

# Support Vector Machine (SVM)

### Convex Optimization (CVXOPT) method

In [14]:
# SVM

class SVM_cvxopt():
    def __init__(self, x_train, y_train, gamma, C):
        self.kernel = Gaussian(gamma)
        self.x_train = x_train
        self.y_train = y_train * 2 - 1
        self.K = self.kernel_matrix(x_train)
        self.C = C
        self.b = None
        self.lam = None
                
    def distance(self, x):
        return cdist(self.x_train, x, metric='euclidean')
    
    def kernel_matrix(self, x):
        return self.kernel(self.distance(x))
        
    def fit(self):
        L = self.x_train.shape[0]
        P = matrix(np.outer(self.y_train, self.y_train) * self.K)
        q = matrix(-np.ones(self.x_train.shape[0]))
        G = matrix(np.vstack((-np.eye(L), np.eye(L))))
        h = matrix(np.hstack((np.zeros(L), np.ones(L) * self.C)))
        A = matrix(self.y_train.reshape(1, L), tc='d')
        b = matrix(np.zeros(1))
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        self.lam = np.array(sol['x']).reshape(-1) 
        self.b = np.mean(self.y_train - (self.lam * self.y_train) @ self.K)
        return sol['status'], sol['iterations']
        
    def predict(self, x):
        return (np.sign((self.lam * self.y_train) @ self.kernel_matrix(x) + self.b) == 1).astype(int)
    
    def accuracy(self, x, y):
        return (self.predict(x) == y).mean()
        

In [ ]:
# Cross-validation !! to add time

models_data = []
for C in tqdm(range(1, 9, 4)):
    for gamma in tqdm(range(1, 9, 4)):
        kf = KFold(n_splits=3, shuffle=True)
        kf.get_n_splits(x_train_val)

        for train_index, valid_index in kf.split(x_train_val):

            x_train = x_train_val[train_index]
            y_train = y_train_val[train_index]
            x_val = x_train_val[valid_index]
            y_val = y_train_val[valid_index]

            start = time.time()
            model = SVM_cvxopt(x_train, y_train, gamma, C)
            logging.warning('fitting...')
            status, nit = model.fit()
            end = time.time()

            train_acc = model.accuracy(x_train, y_train)
            val_acc = model.accuracy(x_val, y_val)
            test_acc = model.accuracy(x_test, y_test)

            models_data.append({'C': C,
                                'gamma': gamma,
                                'train_acc': train_acc,
                                'val_acc': val_acc,
                                'test_acc': test_acc,
                                'number of iterations': nit,
                                'KKT conditions': status,
                                'time': end - start})

In [ ]:
# Results

models_data = pd.DataFrame(models_data)
models_data

In [ ]:
# Best model

grouped = models_data.groupby(['N', 'sigma']).agg('mean').reset_index()
idx = grouped.val_acc.idxmin()
grouped.loc[idx]

### Most Violating Pair (MVP) decomposition method

In [ ]:
# SVM

class SVM_mvp():
    def __init__(self, x_train, y_train, gamma, C):
        self.kernel = Gaussian(gamma)
        self.x_train = x_train
        self.y_train = y_train * 2 - 1
        self.K = self.kernel_matrix(x_train)
        self.C = C
        self.b = None
        self.lam = None
                
    def distance(self, x):
        return cdist(self.x_train, x, metric='euclidean')
    
    def kernel_matrix(self, x):
        return self.kernel(self.distance(x))
        
    def fit(self):
        L = self.x_train.shape[0]
        P = matrix(np.outer(self.y_train, self.y_train) * self.K)
        q = matrix(-np.ones(self.x_train.shape[0]))
        G = matrix(np.vstack((-np.eye(L), np.eye(L))))
        h = matrix(np.hstack((np.zeros(L), np.ones(L) * self.C)))
        A = matrix(self.y_train.reshape(1, L), tc='d')
        b = matrix(np.zeros(1))
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        self.lam = np.array(sol['x']).reshape(-1) 
        self.b = np.mean(self.y_train - (self.lam * self.y_train) @ self.K)
        return sol['status'], sol['iterations']
        
    def predict(self, x):
        return (np.sign((self.lam * self.y_train) @ self.kernel_matrix(x) + self.b) == 1).astype(int)
    
    def accuracy(self, x, y):
        return (self.predict(x) == y).mean()
        